In [4]:
import warnings
warnings.filterwarnings('ignore')

In [31]:

from tensorflow.keras.models import load_model
import pickle
import librosa
import numpy as np


In [32]:
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])
    
    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [33]:
loaded_model = load_model('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/total_best_model.keras')

/home/badri/mine/ser/gnd/ser_env/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 28 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [34]:
with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_scaler2.pickle', 'rb') as f:
    new_scaler2 = pickle.load(f)


with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_encoder2.pickle', 'rb') as f:
    new_encoder2 = pickle.load(f)

In [35]:
def get_predict_feat(path):
    #d, s_rate= librosa.load(path, duration=2.5, offset=0.6)
    d, s_rate= load_fixed_audio(path)
    #librosa.load(path, duration=2.5, offset=0.6)
    res=extract_features(d)
    result=np.array(res)
    result=np.reshape(result,newshape=(1,2376))
    i_result = new_scaler2.transform(result)
    final_result=np.expand_dims(i_result, axis=2)
    
    return final_result

In [82]:
emotions1={1:'Neutral', 2:'Calm', 3:'Happy', 4:'Sad', 5:'Angry', 6:'Fear', 7:'Disgust',8:'Surprise'}
def prediction(path1):
    res=get_predict_feat(path1)
    predictions=loaded_model.predict(res)
    y_pred = new_encoder2.inverse_transform(predictions)
    return y_pred[0][0]

In [37]:
def load_fixed_audio(path, target_duration=2.5):
    y, sr = librosa.load(path,duration=target_duration, offset=0.1)
    if len(y) < target_duration * sr:
        pad_length = int(target_duration * sr) - len(y)
        y = np.pad(y, (0, pad_length))  # Pad with zeros
    return y, sr


In [49]:
import os
total = os.listdir('/home/badri/mine/ser/datasets/crema-d/AudioWAV/')

In [62]:
output_dict = {'DIS' : 'disgust', 'HAP':'happy', 'ANG':'angry','SAD':'sad','FEA':'fear', 'NEU' :' neutral' }

In [65]:
#sub 
print(output_dict[sub[2][9:12]])
print(sub[2][9:12])

disgust
DIS


In [68]:
target_var = []
for sub in total:
    target_var.append(output_dict[sub[9:12]])
    

In [73]:
sub

'1067_ITH_NEU_XX.wav'

In [74]:
prediction(f'/home/badri/mine/ser/datasets/crema-d/AudioWAV/{sub}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
fear


In [ ]:
%%time
total_len = len(total)
score = 0
for i, file in enumerate(total):
    pred = prediction(f'/home/badri/mine/ser/datasets/crema-d/AudioWAV/{file}')
    actual = target_var[i]
    
    if pred == actual:
        score += 1
    
print(f"{int((score/total_len) * 100)}%")

        


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━

In [90]:
7/10

0.7

In [ ]:

import sounddevice as sd
import numpy as np
from tensorflow.keras.models import load_model
import pickle
import librosa

def record_audio(duration=2.5, sr=22050):
    """Records audio for a given duration and sample rate."""
    print("Recording audio in memory...")
    audio_data = sd.rec(int(duration * sr), samplerate=sr, channels=1, dtype='float32')
    sd.wait()  # Wait until recording is finished
    audio_data = audio_data.flatten()  # Flatten the array to 1D
    return audio_data, sr



def process_audio_in_memory(audio_array, sr, target_duration=2.5):
    """Ensures the audio is exactly target_duration seconds long (pads if necessary)."""
    required_length = int(target_duration * sr)
    if len(audio_array) < required_length:
        pad_length = required_length - len(audio_array)
        audio_array = np.pad(audio_array, (0, pad_length))
    return audio_array, sr

def zcr(data, frame_length, hop_length):
    zcr_feature = librosa.feature.zero_crossing_rate(data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(zcr_feature)

def rmse(data, frame_length=2048, hop_length=512):
    rmse_feature = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(rmse_feature)

def mfcc(data, sr, frame_length=2048, hop_length=512, flatten: bool=True):
    mfcc_feature = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)  # n_mfcc can be adjusted
    # Transpose so that time is on axis 0
    mfcc_feature = mfcc_feature.T
    return np.ravel(mfcc_feature) if flatten else np.squeeze(mfcc_feature)

def extract_features(data, sr=22050, frame_length=2048, hop_length=512):
    """Extracts features by combining ZCR, RMSE, and MFCC."""
    features = np.hstack((
        zcr(data, frame_length, hop_length),
        rmse(data, frame_length, hop_length),
        mfcc(data, sr, frame_length, hop_length)
    ))
    return features




# Load the pre-trained model, scaler, and encoder.
loaded_model = load_model('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/total_best_model.keras')

with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_scaler2.pickle', 'rb') as f:
    new_scaler2 = pickle.load(f)

with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_encoder2.pickle', 'rb') as f:
    new_encoder2 = pickle.load(f)

def get_predict_feat_from_array(audio_array, sr):
    # Ensure the audio is exactly 2.5 seconds long.
    audio_array, sr = process_audio_in_memory(audio_array, sr, target_duration=2.5)
    # Extract features
    features = extract_features(audio_array, sr=sr)
    result = np.array(features)
    # Reshape according to your model input (example shape: (1, 2376))
    result = np.reshape(result, newshape=(1, 2376))
    # Scale features
    scaled_result = new_scaler2.transform(result)
    # Expand dimensions if necessary (example adds a channel dimension)
    final_result = np.expand_dims(scaled_result, axis=2)
    return final_result

def prediction_from_array(audio_array, sr):
    features = get_predict_feat_from_array(audio_array, sr)
    predictions = loaded_model.predict(features)
    y_pred = new_encoder2.inverse_transform(predictions)
    print("Predicted Emotion:", y_pred[0][0])



    



[NbConvertApp] Converting notebook testing_pretrained.ipynb to python
[NbConvertApp] Writing 4810 bytes to testing_pretrained.py


import pickle
from tensorflow.keras.models import Sequential, model_from_json


import librosa
import numpy as np



json_file = open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json, custom_objects={"Sequential": Sequential})
print("Model loaded successfully from JSON")

loaded_model.summary()

loaded_model.load_weights('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/best_model1_weights.h5')
print("loaded succesfully")

loaded_model.summary()

loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


type(encoder2)

prediction("/home/badri/mine/ser/datasets/revdess/Actor_21/03-01-04-02-02-02-21.wav")


import pandas
emotions = pandas.read_csv('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/y_label.csv')


Y=emotions['Emotions'].values

from sklearn.preprocessing import OneHotEncodermimport numpy as np

encoder = OneHotEncoder(sparse_output=False)  # Use sparse_output instead of sparse
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1))


with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_encoder2.pickle', 'wb') as f:
    pickle.dump(encoder, f)

with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_encoder2.pickle', 'rb') as f:
    new_encoder2 = pickle.load(f)



prediction('/home/badri/mine/ser/datasets/test_audio/shock.wav')

loaded_model.save('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/total_best_model.keras')

model.summary()

from sklearn.preprocessing import StandardScaler
new_scaler2 = StandardScaler()

new_scaler2.mean_ = old_scaler.mean_
new_scaler2.scale_ = old_scaler.scale_
new_scaler2.var_ = old_scaler.var_  # If available in the old version
new_scaler2.n_features_in_ = old_scaler.n_features_in_

import pickle

with open('/home/badri/mine/ser/saved_models/kaggle_saved_gpu3hrs/new_scaler2.pickle', 'wb') as f:
    pickle.dump(new_scaler2, f)